# Detecting Features in Images

In [1]:
import tensorflow as tf
data =  tf.keras.datasets.fashion_mnist

2024-11-13 13:21:30.759680: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 13:21:30.813731: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 13:21:30.814874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 13:21:32.027086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
(training_images, training_labels), (test_images, test_lables) = data.load_data()
training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images / 255.0

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

2024-11-13 10:37:43.167164: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
   4/1875 [..............................] - ETA: 42s - loss: 0.2383 - accuracy: 0.9141

2024-11-13 10:39:59.041195: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 [==============================] - 44s 23ms/step - loss: 0.2645 - accuracy: 0.9029
Epoch 2/5
1875/1875 [==============================] - 40s 21ms/step - loss: 0.2287 - accuracy: 0.9153
Epoch 3/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.1995 - accuracy: 0.9258
Epoch 4/5
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1765 - accuracy: 0.9341
Epoch 5/5
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1564 - accuracy: 0.9402


In [12]:
model.evaluate(test_images, test_lables)

313/313 [==============================] - 2s 6ms/step - loss: 0.2775 - accuracy: 0.8993


[0.27753880620002747, 0.8992999792098999]

In [13]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_lables[0])


313/313 [==============================] - 2s 5ms/step
[2.6980675e-07 2.4123286e-09 2.3320785e-09 4.4448800e-09 1.4464906e-10
 4.7101175e-06 8.6295221e-10 1.2947056e-06 3.2780804e-07 9.9999326e-01]
9


### Building a CNN to Distinguish Between Horses and Humans

#### The Keras ImageDataGenerator

In [29]:
import urllib.request
import zipfile

training_url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = '../data/horse-or-human/training/'
urllib.request.urlretrieve(training_url, training_file_name)

zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

KeyboardInterrupt: 

In [30]:
from tensorflow.keras.preprocessing.image import  ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
training_dir = '../data/horse-or-human/training/'

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


#### CNN Architecture for Horses or Humans

In [3]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(300, 300, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(300, 300, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(300, 300, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.5),  # Add dropout to reduce overfitting
    tf.keras.layers.Dense(1, activation='sigmoid')
])

2024-11-13 13:21:38.628930: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-13 13:21:38.735206: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43655168 exceeds 10% of free system memory.
2024-11-13 13:21:38.752361: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43655168 exceeds 10% of free system memory.
2024-11-13 13:21:38.761309: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43655168 exceeds 10% of free system memory.


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 8)       224       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 8)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 16)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 85264)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [6]:
# Assuming train_generator is already defined
history = model.fit(train_generator, epochs=10)


Epoch 1/10


2024-11-13 13:21:41.718756: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43655168 exceeds 10% of free system memory.
2024-11-13 13:21:42.210960: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43655168 exceeds 10% of free system memory.


33/33 [==============================] - 25s 720ms/step - loss: 1.6606 - accuracy: 0.6641
Epoch 2/10
33/33 [==============================] - 26s 767ms/step - loss: 0.3585 - accuracy: 0.8617
Epoch 3/10
33/33 [==============================] - 25s 738ms/step - loss: 0.1831 - accuracy: 0.9299
Epoch 4/10
33/33 [==============================] - 25s 744ms/step - loss: 0.1047 - accuracy: 0.9679
Epoch 5/10
33/33 [==============================] - 24s 732ms/step - loss: 0.0477 - accuracy: 0.9864
Epoch 6/10
33/33 [==============================] - 25s 757ms/step - loss: 0.0290 - accuracy: 0.9922
Epoch 7/10
33/33 [==============================] - 28s 841ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 8/10
33/33 [==============================] - 27s 819ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 9/10
33/33 [==============================] - 27s 808ms/step - loss: 0.0517 - accuracy: 0.9834
Epoch 10/10
33/33 [==============================] - 26s 792ms/step - loss: 0.0046 - accuracy: 1.0000


#### Adding Validation to the Horses or Humans Dataset

In [13]:
validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [14]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = train_datagen.flow_from_directory(
validation_dir,
target_size=(300, 300),
class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [16]:
history = model.fit_generator(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

/tmp/ipykernel_1251173/1497470093.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
33/33 [==============================] - 27s 820ms/step - loss: 1.1486e-04 - accuracy: 1.0000 - val_loss: 2.0980 - val_accuracy: 0.8281
Epoch 2/10
33/33 [==============================] - 28s 832ms/step - loss: 4.6620e-05 - accuracy: 1.0000 - val_loss: 2.4031 - val_accuracy: 0.8164
Epoch 3/10
33/33 [==============================] - 26s 793ms/step - loss: 7.2739e-05 - accuracy: 1.0000 - val_loss: 2.1717 - val_accuracy: 0.8320
Epoch 4/10
33/33 [==============================] - 27s 802ms/step - loss: 2.9631e-05 - accuracy: 1.0000 - val_loss: 2.1040 - val_accuracy: 0.8438
Epoch 5/10
33/33 [==============================] - 27s 798ms/step - loss: 5.0797e-05 - accuracy: 1.0000 - val_loss: 2.1125 - val_accuracy: 0.8477
Epoch 6/10
33/33 [==============================] - 26s 808ms/step - loss: 2.2294e-05 - accuracy: 1.0000 - val_loss: 2.1332 - val_accuracy: 0.8477
Epoch 7/10
33/33 [==============================] - 25s 765ms/step - loss: 9.9980e-06 - accuracy: 1.0000 - val_loss: 2

#### Testing Horse or Human Images

In [22]:
from keras.preprocessing import image
import numpy as np

path = '../data/download.jpeg'
img = image.load_img(path, target_size=(300, 300))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

image_tensor = np.vstack([x])
classes = model.predict(image_tensor)

if classes[0]>0.5:
    print(path + " is a human")
else:
    print(path + " is a horse")

1/1 [==============================] - 0s 24ms/step
../data/download.jpeg is a horse


### Image Augmentation

- create additional new data by amending what it has using a number of transforms

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)

train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

### Transfer Learning

In [31]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"

urllib.request.urlretrieve(weights_url, weights_file)
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
include_top=False,
weights=None)

pre_trained_model.load_weights(weights_file)

In [33]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_96 (Conv2D)          (None, 74, 74, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 74, 74, 32)           96        ['conv2d_96[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 74, 74, 32)           0         ['batch_normalizati

In [34]:
for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print(f'last layer output shape: {last_layer.output_shape}')
last_output = last_layer.output

last layer output shape: (None, 7, 7, 768)


In [38]:
from tensorflow.keras import layers


# Flatten the output layer to 1 dimension
flatten_layer = tf.keras.layers.Flatten()  # instantiate the layer
x = flatten_layer(x)       # call it on the given tensor

# Add a fully connected layer with 1, 024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

In [39]:
from tensorflow.keras import Model

model = Model(pre_trained_model.input, x)

# model.compile(optimizer=RMSprop(learning_rate=0.0001),
#               loss='binary_crossentropy',
#               metrics=['acc'])

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: [[0.]]

### Multiclass Classification

In [44]:
!wget --no-check-certificate \
https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
-O /tmp/rps.zip
local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()
TRAINING_DIR = "/tmp/rps/"
training_datagen = ImageDataGenerator(
rescale = 1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

--2024-11-13 15:12:41--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.171.251, 142.250.200.251, 142.250.201.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.171.251|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-13 15:12:43 ERROR 404: Not Found.



BadZipFile: File is not a zip file

In [ ]:
train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical'
)

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image:
    # 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',
    input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',
metrics=['accuracy'])